In [1]:
import numpy
from pandas import DataFrame
from pandas import Series
from pandas import ExcelFile
# 추가
from pandas import concat # 행 단위 병합 기능을 제공하는 합수
from pandas import merge  # 열 단위 병합 기능을 제공하는 함수

## 중복되는 데이터가 존재하는 경우의 열 단위 병합

In [2]:
df_first = DataFrame({'아이디' : ['hello', 'world', 'python', 'hello'],
                      '결제금액' : [14000, 13000, 15000, 13000]})
df_first

,아이디,결제금액
0,hello,14000
1,world,13000
2,python,15000
3,hello,13000


In [3]:
df_second = DataFrame({'아이디' : ['hello', 'python', 'python', 'world'],
                      '결제금액' : [300, 500, 100, 200]})
df_second

,아이디,결제금액
0,hello,300
1,python,500
2,python,100
3,world,200


In [4]:
# on='아이디' : 아이디명을 기준으로 병합한다.
merge(df_first, df_second, on='아이디')

,아이디,결제금액_x,결제금액_y
0,hello,14000,300
1,world,13000,200
2,python,15000,500
3,python,15000,100
4,hello,13000,300


### 컬럼명이 같으면 데이터가 다 들어간다.

## =========================================================

## 공통 컬럼이 두 개이상 존재하는 경우의 열 단위 병합

In [5]:
df_a = DataFrame({ 
                    '고객명' : ['민수', '수영'],
                    '데이터' : ['20000', '10000'],
                    '날짜'   : ['2024-01-01', '2024-01-01']
                 })
df_a

,고객명,데이터,날짜
0,민수,20000,2024-01-01
1,수영,10000,2024-01-01


In [6]:
df_b = DataFrame({ 
                    '고객명' : ['민수', '수영'],
                    '데이터' : ['21세', '20세']
                 })
df_b

,고객명,데이터
0,민수,21세
1,수영,20세


### 1. merge 처리하기
> 고객 컬럼과 데이터 컬럼이 동시에 일치하는 데이터를 찾아서 병합한다.

In [7]:
# 존재하지 않으므로 결과는 없다.
merge(df_a, df_b)

,고객명,데이터,날짜


### 2. `on` 파라미터를 사용하여 기준 열을 지정하자
> df_a 의 데이터는 `금액`을 의미하지만, df_b 의 데이터는 `나이` 를 의미하므로<br>
병합의 기준으로 사용될 수 없다.

In [8]:
# 기준 열이 아니면서 이름이 같은 열에는 _x 또는 _y 와 같은 접미사가 붙는다.
merge(df_a, df_b, on='고객명')

,고객명,데이터_x,날짜,데이터_y
0,민수,20000,2024-01-01,21세
1,수영,10000,2024-01-01,20세


### 3. 컬럼명을 수정하자
> 데이터_x 는 금액으로, 데이터_y 는 나이로

In [9]:
m_res = merge(df_a, df_b, on='고객명')
m_res2 = m_res.rename(columns={'데이터_x' : '금액', '데이터_y' : '나이'})
m_res2

,고객명,금액,날짜,나이
0,민수,20000,2024-01-01,21세
1,수영,10000,2024-01-01,20세


In [10]:
m_res2.reindex(columns=['고객명', '나이', '금액', '날짜'])

,고객명,나이,금액,날짜
0,민수,21세,20000,2024-01-01
1,수영,20세,10000,2024-01-01


## =====================================================

## 공통 컬럼이 존재하지 않는 경우의 열 단위 병합

### `left_on` , `right_on` 파라미터를 사용하여 병합의 기준이 되는 열 이름을 명시해야한다.

In [11]:
국어점수 = DataFrame({'이름': ['영희','철수'],'국어':[87,91]})
국어점수

,이름,국어
0,영희,87
1,철수,91


In [12]:
영어점수 = DataFrame({'성명': ['영희','철수'],'영어':[90,82]})
영어점수

,성명,영어
0,영희,90
1,철수,82


### 1. 국어의 이름 컬럼과 영어의 성명 컬럼이 같은 데이터를 병합 

In [13]:
국영 = merge(국어점수, 영어점수, left_on=['이름'], right_on=['성명'])
국영

,이름,국어,성명,영어
0,영희,87,영희,90
1,철수,91,철수,82


### 2. 중복되는 값을 가지는 `성명` 컬럼은 필요없기 때문에 삭제하자

In [14]:
국영.drop('성명', axis=1, inplace=True)
국영

,이름,국어,영어
0,영희,87,90
1,철수,91,82


## ===================================================

## 인덱스를 기준으로 병합
### `left_index=True` , `right_index=True` 파라미터를 선택적으로 사용한다.

In [15]:
수학점수 = DataFrame({'수학':[90,82]}, index=['민철','봉구'])
수학점수

,수학
민철,90
봉구,82


In [16]:
과학점수 = DataFrame({'과학':[90,82]}, index=['민철','철수'])
과학점수

,과학
민철,90
철수,82


### 1. 중복되어 있는 민철의 데이터만 뽑아오자

In [17]:
수과 = merge(수학점수, 과학점수, left_index=True, right_index=True)
수과

,수학,과학
민철,90,90


### 2. 모든 데이터 뽑아오기

In [18]:
수과2 = merge(수학점수, 과학점수, left_index=True, right_index=True, how='outer')
수과2

,수학,과학
민철,90.0,90.0
봉구,82.0,NaN
철수,NaN,82.0


## ==============================================================

## 왼쪽에서는 인덱스 사용 , 오른쪽에서는 이름 사용

### 인덱스가 사라지고 모든 컬럼으로 지정된다.

In [19]:
한국사 = DataFrame({'한국사':[87,91]}, index=['영희','철수'])
한국사

,한국사
영희,87
철수,91


In [20]:
세계사 = DataFrame({'세계사':[90,82], '이름':['영희','철수']})
세계사

,세계사,이름
0,90,영희
1,82,철수


In [21]:
한세 = merge(한국사, 세계사, left_index=True, right_on=['이름'])
한세

,한국사,세계사,이름
0,87,90,영희
1,91,82,철수


### 1. 새롭게 인덱스로 지정할 데이터 추출

In [24]:
이름 = list(한세['이름'])
이름

['영희', '철수']

### 2. { before : after } 형식의 딕셔너리로 구성 

In [25]:
indexDic = {}
for i, v in enumerate(이름) :
    indexDic[i] = v
    
indexDic

{0: '영희', 1: '철수'}

### 3. 인덱스를 변경하자

In [27]:
한세.rename(index=indexDic, inplace=True)
한세

,한국사,세계사,이름
영희,87,90,영희
철수,91,82,철수


### 4. 이름 컬럼 삭제하자

In [28]:
한세.drop(['이름'], axis=1, inplace=True)
한세

,한국사,세계사
영희,87,90
철수,91,82
